In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# ruff: noqa: E402
# find the root of the project
import os
from pathlib import Path
import sys
import polars as pl
import dotenv


ROOT = Path(os.getcwd()).parent
while not ROOT.joinpath(".git").exists():
    ROOT = ROOT.parent

# add the root to the python path
sys.path.append(str(ROOT))


dotenv.load_dotenv(ROOT.joinpath(".env"))

from src.utils import check_gpu_available

GPU = check_gpu_available()
print(f"GPU available: {GPU}")

GPU available: True


In [4]:
import geopandas as gpd

keep_lanes = gpd.read_file(ROOT / "data/calibration_lanes_new.geojson")
keep_lanes = keep_lanes.loc[keep_lanes['name'].str.contains('Walmart'), 'name'].values

In [5]:
import geopandas as gpd
from src.geometry import RoadNetwork

LANE_WIDTH = 3.55
net = RoadNetwork(
    lane_gdf=gpd.read_file(ROOT / "data/calibration_lanes_new.geojson"),
    # keep_lanes=keep_lanes,
    step_size=0.05,
)

In [6]:
net.lane_gdf.explore()

In [7]:
from datetime import timedelta
import polars as pl
from src.radar import CalibratedRadar
from src.pipelines.open_file import prep_df


USE_FRONT = False


radar_obj = CalibratedRadar(
    radar_location_path=ROOT / "configuration" / "march_calibrated.yaml",
)

In [8]:
import numpy as np


radar_df = (
    pl.scan_parquet(
        Path("/home/max/Development/ua-traffic-data/tmp/all_working.parquet")
    )
    .with_columns(
        pl.col("epoch_time").dt.replace_time_zone("UTC").dt.round("100ms"),
    )
    .pipe(prep_df, f=radar_obj)
    .collect(streaming=True)
    .pipe(
        net.map_to_lane,
        dist_upper_bound=(LANE_WIDTH / 2) + 0.5,  # centered on one of the lanes,
        utm_x_col="utm_x",
        utm_y_col="utm_y",
    )
    .rename(
        {
            "name": "lane",
            "angle": "heading_lane",
        }
    )
    .with_columns((pl.col("heading_utm") - pl.col("heading_lane")).alias("angle_diff"))
    .with_columns(
        # find the portion of velocity that is in the direction of the lane
        pl.arctan2(pl.col("angle_diff").sin(), pl.col("angle_diff").cos()).alias(
            "angle_diff"
        )
    )
    .with_columns(
        pl.when(
            pl.col("angle_diff").abs() > np.deg2rad(30),
        )
        .then(pl.lit(None))
        .otherwise(pl.col("angle_diff"))
        .interpolate()
        .over(["object_id", "lane"])
        .alias("angle_diff")
    )
    .filter(pl.col("angle_diff").is_not_null())
)

function: create_object_id took: 0.0001308917999267578 seconds
function: filter_short_trajectories took: 0.000392913818359375 seconds
function: clip_trajectory_end took: 0.0002071857452392578 seconds
function: resample took: 0.0002276897430419922 seconds
function: fix_duplicate_positions took: 0.00013589859008789062 seconds
function: set_timezone took: 4.38690185546875e-05 seconds
function: add_cst_timezone took: 2.7418136596679688e-05 seconds
function: add_heading took: 2.86102294921875e-05 seconds
function: rotate_radars took: 0.0004978179931640625 seconds
function: update_origin took: 0.0004878044128417969 seconds


In [9]:
# from h3ronpy.polars.vector import coordinates_to_cells
# from h3ronpy.arrow import cells_to_string

# h3_df = (
#     pl.scan_parquet(
#         Path("/home/max/Development/ua-traffic-data/tmp/all_working.parquet")
#     )
#     .with_columns(
#         pl.col("epoch_time").dt.replace_time_zone("UTC").dt.round("100ms"),
#     )
#     .pipe(prep_df, f=radar_obj)
#     .pipe(radar_obj.add_cst_timezone)
#     .filter(
#         (pl.col('epoch_time_cst').dt.hour() == 7) & (pl.col('epoch_time_cst').dt.minute() < 10)
#     )
#     .collect()
#     .pipe(
#         lambda x: x.with_columns(
#             h3=pl.Series(cells_to_string(coordinates_to_cells(x["lat"], x["lon"], 15)))
#         )
#     )
#     # .group_by_dynamic("epoch_time", every="1s", by="h3")
#     # .agg(pl.col("object_id").n_unique())
# )

In [10]:
radar_df = (
    radar_df.filter(pl.col("lane").is_not_null())
    .sort("epoch_time")
    .with_columns(pl.col("s").diff().sum().over("object_id", "lane").alias("lane_dist"))
)

In [11]:
# radar_df.pivot(
#     columns=["lane"], values="lane_dist", index="object_id", aggregate_function="last"
# ).describe()

In [12]:
filtered_radar_df = (
    radar_df.filter(pl.col("lane_dist") > 0)
    .join(
        pl.from_pandas(
            net.lane_gdf.copy()
            .assign(length=lambda x: x.geometry.length)
            .drop(["geometry", "id"], axis=1)
        ).rename({"name": "lane"}).with_columns(
            pl.col('lane').str.split("_").list.get(0).alias('signal')
        ),
        on="lane",
    )
    .with_columns((pl.col("lane_dist") / pl.col("length")).alias("lane_coverage"))
    .pipe(radar_obj.add_cst_timezone)
    .filter(pl.col("epoch_time_cst").dt.day() == 13)
    # .filter(pl.col("lane_coverage") > 0.3)
)

function: add_cst_timezone took: 0.000263214111328125 seconds


In [13]:
filtered_radar_df.select(['object_id', 'epoch_time_cst', 'lane', 'lane_coverage', 'ip']).write_parquet(ROOT / "data" / "detections.parquet")

In [25]:
plot_df = (
    filtered_radar_df
    .lazy()
    .group_by_dynamic("epoch_time", every="1m", by=["ip", "lane", 'signal'])
    .agg(pl.col("object_id").n_unique().alias("volume"))
    .pipe(radar_obj.add_cst_timezone)
    .filter(pl.col("epoch_time_cst").dt.day() == 13)
    .group_by("lane", 'epoch_time_cst')
    .agg(pl.col('volume').max())
    .sort("epoch_time_cst", "lane")
    .collect()
    .upsample(
        'epoch_time_cst',
        every='1m',
        by=['lane']
    )
    
    .with_columns(
        # pl.col('volume').fill_null(0),
        pl.col('lane').forward_fill(),
        pl.col('volume').fill_null(0)
        
    )

)
plot_df.head()

function: add_cst_timezone took: 6.031990051269531e-05 seconds


epoch_time_cst,lane,volume
"datetime[ms, US/Central]",str,u32
2023-03-13 00:11:00 CDT,"""Walmart_WBL""",1
2023-03-13 00:12:00 CDT,"""Walmart_WBL""",0
2023-03-13 00:13:00 CDT,"""Walmart_WBL""",0
2023-03-13 00:14:00 CDT,"""Walmart_WBL""",0
2023-03-13 00:15:00 CDT,"""Walmart_WBL""",0


In [27]:
plot_df.write_parquet(ROOT / 'data' / 'radar_detections.parquet')

In [ ]:
pl.DataFrame().upsample()

In [18]:
import plotly.graph_objects as go


for signal, df in plot_df.group_by('signal'):
    fig = go.Figure()

    for lane, detector_df in df.groupby("lane",  maintain_order=True):
        fig.add_trace(
            go.Scatter(
                x=detector_df["epoch_time_cst"].cast(str),
                y=detector_df["volume"],
                name=f"{lane}",
                mode="lines",
                # line=dict(color=color_dict[detector_df[0]]),
            )
        )
    
    fig.update_layout(
        # title=f"Signal {tl_df[0]}",
        yaxis_title="Volume",
        xaxis_title="Time",
        legend_title="Detector",
        font=dict(family="Courier New, monospace", size=18, color="RebeccaPurple"),
        # yaxis=dict(
        #     # set the range of the xaxis to the range of the data
        #     range=[0, 1000],
        # ),
    )
    
    fig.show()

/tmp/ipykernel_922327/2053640720.py:7: DeprecationWarning:

`groupby` is deprecated. It has been renamed to `group_by`.

